In [1]:
import numpy as np
import pandas as pd
from lightgbm import LGBMRegressor


In [2]:
df_train_raw = pd.read_csv("data/train.csv")
df_train_raw.head(5)

,index,송하인_격자공간고유번호,수하인_격자공간고유번호,물품_카테고리,운송장_건수
0,0,5011000595017300,2871000192069300,음반,3
1,1,4148000690043300,5011000264024400,문화컨텐츠,3
2,2,5011000078068400,1120000007005400,농산물,3
3,3,4127100048006400,5011000587019400,기타식품,7
4,4,5011000078068400,2823700010076300,농산물,3


In [48]:
sample_string = str(df_train_raw.at[0, "송하인_격자공간고유번호"])
len_transfer = len(sample_string)
sample_string = str(df_train_raw.at[0, "수하인_격자공간고유번호"])
len_receive = len(sample_string)

def parser(df, column):
    list_return = []
    for _, x in df[column].iteritems():
        list_temp = []
        for char in str(x):
            list_temp.append(char)
        list_return.append(list_temp)
    return list_return


list_transfer = parser(df_train_raw, "송하인_격자공간고유번호")
list_receive = parser(df_train_raw, "수하인_격자공간고유번호")
list_trans_name = ["Trans_" + str(x) for x in range(len_transfer)]
list_receive_name = ["Receive_" + str(x) for x in range(len_receive)]
df_transfer = pd.DataFrame(list_transfer, columns = list_trans_name)
df_receive = pd.DataFrame(list_receive, columns = list_receive_name)
df_train = pd.concat([df_train_raw, df_transfer, df_receive], axis=1)
df_train.head(5)

,index,송하인_격자공간고유번호,수하인_격자공간고유번호,물품_카테고리,운송장_건수,Trans_0,Trans_1,Trans_2,Trans_3,Trans_4,...,Receive_6,Receive_7,Receive_8,Receive_9,Receive_10,Receive_11,Receive_12,Receive_13,Receive_14,Receive_15
0,0,5011000595017300,2871000192069300,음반,3,5,0,1,1,0,...,0,1,9,2,0,6,9,3,0,0
1,1,4148000690043300,5011000264024400,문화컨텐츠,3,4,1,4,8,0,...,0,2,6,4,0,2,4,4,0,0
2,2,5011000078068400,1120000007005400,농산물,3,5,0,1,1,0,...,0,0,0,7,0,0,5,4,0,0
3,3,4127100048006400,5011000587019400,기타식품,7,4,1,2,7,1,...,0,5,8,7,0,1,9,4,0,0
4,4,5011000078068400,2823700010076300,농산물,3,5,0,1,1,0,...,0,0,1,0,0,7,6,3,0,0


In [49]:
list_name = list_trans_name + list_receive_name
for name in list_name:
    if len(df_train[name].drop_duplicates()) < 2:
        df_train.drop(columns = name, inplace=True)
df_train.head(5)

,index,송하인_격자공간고유번호,수하인_격자공간고유번호,물품_카테고리,운송장_건수,Trans_0,Trans_1,Trans_2,Trans_3,Trans_4,...,Receive_3,Receive_4,Receive_6,Receive_7,Receive_8,Receive_9,Receive_10,Receive_11,Receive_12,Receive_13
0,0,5011000595017300,2871000192069300,음반,3,5,0,1,1,0,...,1,0,0,1,9,2,0,6,9,3
1,1,4148000690043300,5011000264024400,문화컨텐츠,3,4,1,4,8,0,...,1,0,0,2,6,4,0,2,4,4
2,2,5011000078068400,1120000007005400,농산물,3,5,0,1,1,0,...,0,0,0,0,0,7,0,0,5,4
3,3,4127100048006400,5011000587019400,기타식품,7,4,1,2,7,1,...,1,0,0,5,8,7,0,1,9,4
4,4,5011000078068400,2823700010076300,농산물,3,5,0,1,1,0,...,3,7,0,0,1,0,0,7,6,3


In [50]:
list_name_train = df_train.columns[5:]

In [51]:
train_x = df_train[list_name_train].to_numpy()
train_y = df_train["운송장_건수"].to_numpy()
lgbm = LGBMRegressor()
lgbm.fit(train_x, train_y, eval_metric="mse")

LGBMRegressor()

In [52]:
df_test_raw = pd.read_csv("data/test.csv")
list_transfer = parser(df_test_raw, "송하인_격자공간고유번호")
list_receive = parser(df_test_raw, "수하인_격자공간고유번호")
df_transfer = pd.DataFrame(list_transfer, columns = list_trans_name)
df_receive = pd.DataFrame(list_receive, columns=list_receive_name)
print(df_transfer.shape)
df_test = pd.concat([df_test_raw, df_transfer, df_receive], axis=1)
pred = lgbm.predict(df_test[list_name_train].to_numpy())
df_test.shape

(7920, 16)


(7920, 36)

In [53]:
df_submission = pd.DataFrame([[int(x) for x in range(len(pred))], list(pred)]).T
df_submission.columns = ["index", "운송장_건수"]
df_submission["운송장_건수"] = df_submission["운송장_건수"].apply(round)
df_submission["index"] = df_submission["index"].apply(lambda x: ('%0d' % x))
df_submission["운송장_건수"] = df_submission["운송장_건수"].apply(lambda x: ('%0d' % x))
df_submission.head(5)

,index,운송장_건수
0,0,5
1,1,4
2,2,4
3,3,5
4,4,5


In [54]:
df_submission.to_csv("submission.csv", index=False)